In [7]:
from astropy.table import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from astropy.table import hstack
import os
import seaborn as sns
import matplotlib
import matplotlib.backends.backend_pdf
from scipy.stats import chi2, binned_statistic
from astropy.cosmology import Planck13

plt.rc('font', family='serif'), plt.rc('xtick', labelsize=18), plt.rc('ytick', labelsize=18)
plt.rcParams['savefig.dpi'] = 300
plt.rc('text',usetex=True)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
np.set_printoptions(precision=3)

#### Binary Selection

##### useful functions
adapted form Holden et al. 2012

fractional scores (cost functions):\
$s_{sf}=f_c +rf_m - |f_c-rf_m|$\
$s_{q}=rf_c +f_m - |rf_c-f_m|$\
$s = \frac{N_{sf}}{N} \cdot s_{sf} + \frac{N_{q}}{N} \cdot s_{q}$

where $f_c$ and $f_m$ are contamination fraction and missing fraction, $r$ is the factor to emphasize contamination fraction in quiescent sample and missing fraciton in sf sample. \
The goal is to minimize missing fraction and contaminating fraction simultaneously, and can be empahsize on one of them.

#### COSMOS2020 selection

$NUV-r>3(r-J)+1$\
$NUV-r>3.1$

In [3]:
def gv_boundary_2(color1, ic_1, ic_2, slope):
    '''
    color1 can be r-K or r-J
    ic_1, ic_2 are intercepts
    '''
        
    nuv_1 = ic_1
    nuv_2 = slope*color1 + ic_2
    return np.maximum(nuv_1, nuv_2)


def fraction_score(cat,ic_1, ic_2,slope, diagram_type='NUVrK',eval_type='q',verbose=False, factor=3.5):
    '''
    this is the s-score I give to the binary selection box
    for sf and q 
    '''
    
    if diagram_type=='NUVrK':
        color1 = cat['MAG_ABS_r'] - cat['MAG_ABS_Ks']
    elif diagram_type=='NUVrz':
        color1 = cat['MAG_ABS_r'] - cat['MAG_ABS_z']
    elif diagram_type=='NUVrJ':
        color1 = cat['MAG_ABS_r'] - cat['MAG_ABS_J']
    else:
        return 1
    
    delta_gv = cat['MAG_ABS_NUV'] - cat['MAG_ABS_r'] - gv_boundary_2(color1, ic_1, ic_2, slope)
    q = delta_gv > 0
    sf = delta_gv < 0
    true_q = cat['CLASS']==0 
    true_sf = cat['CLASS']!=0
    
    if eval_type=='q':
        FP = sum(true_sf*q)  # false positive
        TP = sum(true_q*q)   # true positive
        FN = sum(true_q*sf)  # false negative
        TN = sum(true_sf*sf) # true negative
    else:
        FP = sum(true_q*sf)  # false positive
        TP = sum(true_sf*sf)   # true positive
        FN = sum(true_sf*q)  # false negative
        TN = sum(true_q*q) # true negative
    if verbose:
        print('confusion matrix',FP,TP,FN,TN)
        
    if TP+FP>0:
        fc = FP/(TP+FP)
    else:
        fc=0.99
    
    if TP+FN>0:
        fm = FN/(TP+FN)
    else:
        fm=0.99
    
    # original formula from Holden et al. 2012
    if eval_type =='sf':
        return fc, fm, fc + factor*fm + abs(fc-factor*fm)
    else:
        return fc, fm, factor*fc + fm + abs(factor*fc-fm)

Next I set up the search grid and do a brute-force calculation of the scores for all points in the search grid

In [9]:
# binary selection box parameter search grid
ic_1_list = np.linspace(2.5, 4.5, 6)  
ic_2_list = np.linspace(1.0, 5.0, 12)
slope_list = np.linspace(1.0, 4.5, 12)

ic_1_mesh, ic_2_mesh, slope_mesh = np.meshgrid(ic_1_list, ic_2_list, slope_list, indexing='ij')
print('search grid size:',len(ic_1_mesh.ravel()))

search grid size: 864


#### load catalog

In [5]:
dev_path = '/Users/lejay/research/lephare_dev/my_code/'
output_dir = dev_path+'output_cats/'
graham_output_dir = dev_path + 'graham_output_cats/'

In [6]:
# the output catalogs (from graham)
name_tag = '_08squdeg'
detect_limit = '_nolimit'
nz_prior = '_nz_prior'
sfq_added = '_sfq_added'
pcat_output_nonir_name = graham_output_dir+'pcat_cat_out'+name_tag+'_nonir_il'+nz_prior+detect_limit+'_formass'+sfq_added+'.fits'
print(pcat_output_nonir_name)

/Users/lejay/research/lephare_dev/my_code/graham_output_cats/pcat_cat_out_08squdeg_nonir_il_nz_prior_nolimit_formass_sfq_added.fits


Next find the selection box that gives lowest score, I do this for 4 difference redshift bins and possibly 2 mass bins\
You can choose NUVrK or NUVrJ

In [10]:
# optimize the selection box
best_box = []
scores_arr = []
sample_size = 1000
diagram_type = 'NUVrJ'

gal_sample = 'pcat'
cat_names = [pcat_output_nonir_name.replace('_sfq_added.fits','c20added.fits')]
zkeyname='Z_COMBINE' # combined phos cat redshift and my redshift

fit_type = 'free' # free, fix_slope, or evolving
r_factors = [1.0,1.0,1.0,1.0]
z_mins = np.array([0.2, 0.5, 0.8, 1.1])
z_maxs = np.array([0.5, 0.8, 1.1, 1.5])

mass_split = ''
if mass_split == 'split':
    mass_cut = 10.3
    mass_lows = [8.0,mass_cut]
    mass_highs = [mass_cut,12.5]
    mass_idx = 1
    mass_low = mass_lows[mass_idx]
    mass_high = mass_highs[mass_idx]
    mass_split_show = '_'+str(mass_low)+'_'+str(mass_high)
else:
    mass_low = 8.0
    mass_high = 12.5
    mass_split_show = ''

print('mass range',mass_low, mass_high)
for cat_name in cat_names:
    cat = Table.read(cat_name)
    cat = cat[(cat['lp_zPDF']>0) & (cat['lp_zPDF']<3)]
    for z_idx in range(len(z_mins)):
        # cut in redshift bin
        cat_z = cat[(cat[zkeyname]>z_mins[z_idx]) & (cat[zkeyname]<z_maxs[z_idx])]
        print(str(z_mins[z_idx])+'<z<'+str(z_maxs[z_idx]))
        
        # sf_weight
        sf_weight = len(cat_z[cat_z['CLASS']==0])/len(cat_z[cat_z['CLASS']==1]) 
        
        # quality flags and photometric flags
        flag_photoz = cat_z['flag_photoz']==1  # internally agreed redshifts between LePhare and EAzY results in COSMOS2020
        flag_optical = cat_z['flag_optical'] 
        flag_irac = cat_z['flag_irac'] 
        flag_nir = cat_z['flag_nir'] 
        phot_config = 'nonir'  # no NIR filters used in LePhare SED fitting (optical 6-band + IRAC)
        cond_filters = (flag_optical>3) & (flag_irac>=1) # minimum of number of filters detected
        
        # mass cut
        cat_z = cat_z[cat_z['MASS_MED_massrun']>mass_low]
        cat_z = cat_z[cat_z['MASS_MED_massrun']<mass_high]
        if len(cat_z)>=sample_size: # use a sub-sample to calibrate when sample size is too large
            select_ids = np.random.choice(np.arange(len(cat_z)), size=sample_size,replace=False)
            cat_z = cat_z[select_ids]
        
        # search the grid to find the best selection box/boundary
        scores = []
        for k in tqdm(range(len(ic_1_mesh.ravel()))):
            fc_sf, fm_sf, score_sf = fraction_score(cat_z, ic_1_mesh.ravel()[k], ic_2_mesh.ravel()[k], slope_mesh.ravel()[k], diagram_type=diagram_type, eval_type='sf',factor=r_factors[z_idx])
            fc_q, fm_q, score_q = fraction_score(cat_z, ic_1_mesh.ravel()[k], ic_2_mesh.ravel()[k], slope_mesh.ravel()[k], diagram_type=diagram_type, eval_type='q',factor=r_factors[z_idx])
            if score_sf >= 0 and score_q>0: 
                scores.append(sf_weight*score_sf + (1-sf_weight)*score_q)
            else:
                scores.append(99.)
        best_id = np.argmin(np.array(scores)) # the selection box with lowest score
        ic_1,ic_2,slope = ic_1_mesh.ravel()[best_id], ic_2_mesh.ravel()[best_id], slope_mesh.ravel()[best_id]
        
        best_box.append([ic_1,ic_2,slope])
        scores_arr.append(scores)
        print('best selection box:',[round(ic_1,3),round(ic_2,3),round(slope,3)],'best score:',round(np.min(scores),4),'avg. score:',round(np.mean(scores),4),'median score:',round(np.median(scores),4),'\n')
        
        # print number of sf/q galaxies
        delta_gv = cat_z['MAG_ABS_NUV'] - cat_z['MAG_ABS_r'] - gv_boundary_2(cat_z['MAG_ABS_r'] - cat_z['MAG_ABS_Ks'], ic_1, ic_2, slope) # vertical deviation from the selection booundary
        q = delta_gv[delta_gv > 0]
        sf = delta_gv[delta_gv < 0]
        print('sf-gals:',len(sf),'q-gals:',len(q))
    
    np.save('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+fit_type+mass_split_show+'.npy',np.array(best_box))
    np.save('scores_lepharecheck_'+diagram_type+'_'+phot_config+'_'+fit_type+mass_split_show+'.npy',np.array(scores_arr))
    

mass range 8.0 12.5
0.2<z<0.5


100%|██████████| 864/864 [01:56<00:00,  7.45it/s]


best selection box: [2.9, 2.091, 1.636] best score: 0.4126 avg. score: 1.3328 median score: 1.5797 

sf-gals: 844 q-gals: 156
0.5<z<0.8


100%|██████████| 864/864 [02:00<00:00,  7.14it/s]


best selection box: [2.5, 2.455, 1.318] best score: 0.4354 avg. score: 1.4273 median score: 1.7046 

sf-gals: 897 q-gals: 103
0.8<z<1.1


100%|██████████| 864/864 [02:49<00:00,  5.10it/s]


best selection box: [3.7, 2.818, 1.318] best score: 0.3285 avg. score: 1.4117 median score: 1.7177 

sf-gals: 937 q-gals: 63
1.1<z<1.5


100%|██████████| 864/864 [01:54<00:00,  7.53it/s]

best selection box: [3.7, 1.364, 2.273] best score: 0.3001 avg. score: 1.3661 median score: 1.5727 

sf-gals: 938 q-gals: 62


#### The calibration plot: 

First row for overall sample and second row for massive galaxies (logM>11.0)


In [ ]:

massive_cut=10.3 # the mass cut for massive galaxy
mass_lows = [8.0, massive_cut]
mass_highs = [massive_cut, 12.5]

diagram_type = 'NUVrJ'
phot_config = 'nonir'
fit_type='free'
massbin_type = 'onebin' # 'onebin' or 'splitbin'

if massbin_type == 'onebin':
    selections_rotation = ['overall','massive']
elif massbin_type == 'splitbin':
    selections_rotation = ['low-mass','massive']
else:
    raise ValueError()
    
fc_list_overall = [] # contatnimate fraction
fm_list_overall = [] # missed fraction
fc_list_massive = [] # contatnimate fraction
fm_list_massive = [] # missed fraction
z_mins = np.array([0.2, 0.5, 0.8, 1.1])
z_maxs = np.array([0.5, 0.8, 1.1, 1.5])

# catalog name
# cat_name = cat_output_no_irac_nir_mass_name
cat_name = graham_output_dir+'pcat_COSMOS_deep_nomaglimit_'+phot_config+'_nz_prior_.fits'
cat_output_c20added = Table.read(cat_name.replace('.fits','c20added.fits'))

fig, axs = plt.subplots(2, 4, figsize = (24, 12), sharey=True)
for row,selection in enumerate(selections_rotation):
    for z_idx,zmin in enumerate(z_mins):
        cat_output_c20added_z = cat_output_c20added[(cat_output_c20added['Z_ML']>zmin) & (cat_output_c20added['Z_ML']<z_maxs[z_idx])]
        
        # COSMOS consistent photoz objects only
        mass_split_show = '_'+str(mass_lows[row])+'_'+str(mass_highs[row])
        if selection=='massive' and massbin_type=='splitbin':
            cond = (cat_output_c20added_z['MASS_MED_massrun']>massive_cut)
            best_box = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+fit_type+mass_split_show+'.npy')[z_idx]
        elif selection=='low-mass':
            cond = (cat_output_c20added_z['MASS_MED_massrun']<massive_cut)
            best_box = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+fit_type+mass_split_show+'.npy')[z_idx]
        else:
            cond = np.ones(len(cat_output_c20added_z)).astype(bool)
            best_box = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+fit_type+'.npy')[z_idx]
            
        ##########################
        # NUVrK(J) selection box 
        print(z_idx, best_box)
        rk_draw = np.linspace(-2,3,100)
        nuv_draw_mid = gv_boundary_2(rk_draw,best_box[0],best_box[1],best_box[2])
        axs[row][z_idx].plot(rk_draw, nuv_draw_mid, color='k')
        
        # COSMOS2020 selection box (NUVrJ)
        if diagram_type == 'NUVrJ':
            rj_draw = np.linspace(-2,3,100)
            nuv_draw_mid = gv_boundary_2(rj_draw, 3.1, 1, 3)
            axs[row][z_idx].plot(rj_draw, nuv_draw_mid, color='b',alpha=0.5)

        # other cuts
        flag_photoz = (cat_output_c20added_z['flag_photoz']==1)
        flag_optical = cat_output_c20added_z['flag_optical'] 
        flag_irac = cat_output_c20added_z['flag_irac'] 
        cond_filters = (flag_optical>2) & (flag_irac>=1)
        cond = cond & flag_photoz & cond_filters
        
        #####################
        # plot data points
        if diagram_type == 'NUVrK':
            im = axs[row][z_idx].scatter(cat_output_c20added_z[cond]['MAG_ABS_r']-cat_output_c20added_z[cond]['MAG_ABS_Ks'], 
                                       cat_output_c20added_z[cond]['MAG_ABS_NUV']-cat_output_c20added_z[cond]['MAG_ABS_r'],
                                       s=2,c=cat_output_c20added_z[cond]['CLASS'],alpha=0.4,cmap='bwr_r')
            axs[row][z_idx].set_xlabel(r'$r-K$',fontsize=21)
        elif diagram_type == 'NUVrJ':
            im = axs[row][z_idx].scatter(cat_output_c20added_z[cond]['MAG_ABS_r']-cat_output_c20added_z[cond]['MAG_ABS_J'], 
                                       cat_output_c20added_z[cond]['MAG_ABS_NUV']-cat_output_c20added_z[cond]['MAG_ABS_r'],
                                       s=2,c=cat_output_c20added_z[cond]['CLASS'],alpha=0.4,cmap='bwr_r')
            axs[row][z_idx].set_xlabel(r'$r-J$',fontsize=21)
        
        axs[row][z_idx].set_ylabel(r'$NUV-r$',fontsize=21)
        axs[row][z_idx].set_xlim([-1.5,3])
        axs[row][z_idx].set_ylim([-1,7])
        axs[row][z_idx].grid()
        axs[row][z_idx].set_title('$'+str(zmin)+'<z<'+str(z_maxs[z_idx])+'$',fontsize=17)
        axs[row][z_idx].annotate(fit_type.replace('_',' '),color='b',xy=([0.03,0.94]),xycoords='axes fraction',fontsize=18)
            
        #### contingency table  ######
        # q=P sf=N
        cat = cat_output_c20added_z[cond]
        cat_sf = cat[cat['CLASS']==1]
        cat_q = cat[cat['CLASS']==0]
        if diagram_type == 'NUVrK':
            rk_sf = np.array(cat_sf['MAG_ABS_r'] - cat_sf['MAG_ABS_Ks'])
            rk_q = np.array(cat_q['MAG_ABS_r'] - cat_q['MAG_ABS_Ks']) 
            delta_gv_sf = cat_sf['MAG_ABS_NUV'] - cat_sf['MAG_ABS_r']-gv_boundary_2(rk_sf,best_box[0],best_box[1],best_box[2])
            delta_gv_q = cat_q['MAG_ABS_NUV'] - cat_q['MAG_ABS_r']-gv_boundary_2(rk_q,best_box[0],best_box[1],best_box[2])
        elif diagram_type == 'NUVrJ':
            rj_sf = np.array(cat_sf['MAG_ABS_r'] - cat_sf['MAG_ABS_J'])
            rj_q = np.array(cat_q['MAG_ABS_r'] - cat_q['MAG_ABS_J']) 
            delta_gv_sf = cat_sf['MAG_ABS_NUV'] - cat_sf['MAG_ABS_r']-gv_boundary_2(rj_sf,best_box[0],best_box[1],best_box[2])
            delta_gv_q = cat_q['MAG_ABS_NUV'] - cat_q['MAG_ABS_r']-gv_boundary_2(rj_q,best_box[0],best_box[1],best_box[2])
        
        sf_sf = len(cat_sf[delta_gv_sf<0])  # TN
        q_q = len(cat_q[delta_gv_q>0])      # TP
        sf_q = len(cat_q[delta_gv_q<0])     # FP
        q_sf = len(cat_sf[delta_gv_sf>0])   # FN
        fc = sf_q/(q_q+sf_q)
        fm = q_sf/(q_q+q_sf)
        if selection == 'massive':
            fc_list_massive.append(fc)
            fm_list_massive.append(fm)
        else:
            fc_list_overall.append(fc)
            fm_list_overall.append(fm)
        
        # inset table
        axs[row][z_idx].annotate('sfc',xy=([0.08,0.24]),xycoords='axes fraction',fontsize=15)
        axs[row][z_idx].annotate('qc',xy=([0.19,0.24]),xycoords='axes fraction',fontsize=15)
        axs[row][z_idx].annotate('sf',xy=([0.03,0.17]),xycoords='axes fraction',fontsize=15)
        axs[row][z_idx].annotate('q',xy=([0.03,0.1]),xycoords='axes fraction',fontsize=15)
        ax_insert2 = axs[row][z_idx].inset_axes([0.08, 0.07, 0.2, 0.15])
        ax_insert2.annotate(str(sf_sf),xy=([-0.9,0.1]),fontsize=14)
        ax_insert2.annotate(str(sf_q),xy=([0.1,0.1]),fontsize=14)
        ax_insert2.annotate(str(q_sf),xy=([-0.9,-0.9]),fontsize=14)
        ax_insert2.annotate(str(q_q),xy=([0.1,-0.9]),fontsize=14)
        ax_insert2.axvline(ymin=-1,ymax=1,x=0,color='k',linewidth=1)
        ax_insert2.axhline(xmin=-1,xmax=1,y=0,color='k',linewidth=1)
        ax_insert2.set_xlim([-1,1])
        ax_insert2.set_ylim([-1,1])
        ax_insert2.set_yticklabels([])
        ax_insert2.set_xticklabels([])


write sfq info in catalog
(i commented out NUVrJ part but I can swtich to it)

In [ ]:
# sfq_info in catalog
# keynames: sfq_nuvrk_myrun_[...]

prefix = 'pcat_'
cat_type = 'all' # all or central_cosmos

if prefix == 'v9pcat_':
    z_keyname = 'Z_COM'
if prefix == 'pcat_' or prefix == 'pcat_extinc_':
    z_keyname = 'Z_COMBINE'
else:
    z_keyname = 'Z_ML'
    
if cat_type == 'all':
    cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep']
elif cat_type == 'central_cosmos':
    cat_names = ['COSMOS_deep']
    
# phot_configs = ['nonirirac','nonir','noirac','allphot']
phot_configs = ['nonir']

# sfq_methods = ['old','free','fix_slope','evolving']
sfq_methods = ['free']
sfq_nuvrj_methods = ['free']

mass_split = '' # '' or 'split'
mass_cut = 10.3
mass_lows = [8.0,mass_cut]
mass_highs = [mass_cut,12.5]
    
for cat_name in cat_names:
    for phot_config in phot_configs:
        if cat_type == 'all': # all or central_cosmos
            cat_full_name = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_formass.fits'  # all fields     
        elif cat_type == 'central_cosmos' and 'pcat' not in prefix:
            cat_full_name = graham_output_dir+'cat_out_08squdeg_'+phot_config+'_il_nz_prior_nolimit_formass.fits' # central cosmos
        elif cat_type == 'central_cosmos' and 'pcat' in prefix:
            cat_full_name = graham_output_dir+'pcat_cat_out_08squdeg_'+phot_config+'_il_nz_prior_nolimit_formass.fits' # pcat central cosmos
            
        cat = Table.read(cat_full_name)
        arr = cat.keys()
        matches = [match for match in arr if "MAG_APER" in match]
        cat.remove_columns(matches)
        print(cat_full_name)
        
        #nuvrk
        diagram_type = 'NUVrK'
        for sfq_fit_type in sfq_methods:
            print(cat_name, phot_config, sfq_fit_type)
            sfq_nuvrk = []
            best_boxes_nuvrk_lowmass = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'_'+str(mass_lows[0])+'_'+str(mass_highs[0])+'.npy')
            best_boxes_nuvrk_highmass = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'_'+str(mass_lows[1])+'_'+str(mass_highs[1])+'.npy')
            best_boxes_mass = [best_boxes_nuvrk_lowmass, best_boxes_nuvrk_highmass]
            best_boxes_nuvrk_overall = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'.npy')
            for i in tqdm(range(len(cat))):
                gal = cat[i]
                if mass_split == 'split' and sfq_fit_type != 'old':
                    selected = False
                    for mass_idx in range(len(mass_lows)):
                        if gal['MASS_MED_massrun']>mass_lows[mass_idx] and gal['MASS_MED_massrun']<=mass_highs[mass_idx]:
                            best_boxes_nuvrk = best_boxes_mass[mass_idx]
                            selected = True
                            continue
                else:
                    best_boxes_nuvrk = best_boxes_nuvrk_overall
                
                if abs(gal['MAG_ABS_r']) < 50. and abs(gal['MAG_ABS_Ks'])< 50. and gal['MAG_ABS_NUV']< 50. and abs(gal['MAG_ABS_r'])< 50.:
                    nuvr = gal['MAG_ABS_NUV'] - gal['MAG_ABS_r']
                    rk = gal['MAG_ABS_r'] - gal['MAG_ABS_Ks']
                
                    if gal[z_keyname]>0.2 and gal[z_keyname]<0.5:
                        delta_gv_nuvrk = nuvr - gv_boundary_2(rk,best_boxes_nuvrk[0][0],best_boxes_nuvrk[0][1],best_boxes_nuvrk[0][2])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.)
                        else:
                            sfq_nuvrk.append(0.)
                    elif gal[z_keyname]>0.5 and gal[z_keyname]<0.8:
                        delta_gv_nuvrk = nuvr - gv_boundary_2(rk,best_boxes_nuvrk[1][0],best_boxes_nuvrk[1][1],best_boxes_nuvrk[1][2])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.)
                        else:
                            sfq_nuvrk.append(0.)
                    elif gal[z_keyname]>0.8 and gal[z_keyname]<1.1:
                        delta_gv_nuvrk = nuvr - gv_boundary_2(rk,best_boxes_nuvrk[2][0],best_boxes_nuvrk[2][1],best_boxes_nuvrk[2][2])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.)
                        else:
                            sfq_nuvrk.append(0.)
                    elif gal[z_keyname]>1.1 and gal[z_keyname]<1.5:
                        delta_gv_nuvrk = nuvr - gv_boundary_2(rk,best_boxes_nuvrk[3][0],best_boxes_nuvrk[3][1],best_boxes_nuvrk[3][2])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.)
                        else:
                            sfq_nuvrk.append(0.)
                    else:
                        sfq_nuvrk.append(-99.)
                else:
                    sfq_nuvrk.append(99.)
                
            sfq_col_nuvrk = Column(name='sfq_nuvrk_myrun_'+sfq_fit_type,data=sfq_nuvrk)  # 1=sf, 0=q
            if 'sfq_nuvrk_myrun'+sfq_fit_type in cat.keys():
                cat.remove_column('sfq_nuvrk_myrun'+sfq_fit_type)
            cat.add_column(sfq_col_nuvrk)
            
        # nuvrj 
#         diagram_type = 'NUVrJ'
#         for sfq_fit_type in sfq_nuvrj_methods:
#             print(cat_name, phot_config, sfq_fit_type)
#             sfq_nuvrj = []
#             for i in tqdm(range(len(cat))):
#                 gal = cat[i]
                
#                 # load selection box
#                 if mass_split == 'split' and sfq_fit_type != 'old':
#                     selected = False
#                     for mass_idx in range(len(mass_lows)):
#                         if gal['MASS_MED_massrun']>mass_lows[mass_idx] and gal['MASS_MED_massrun']<mass_highs[mass_idx]:
#                             best_boxes_nuvrj = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'_'+str(mass_low)+'_'+str(mass_high)+'.npy')
#                             selected = True
#                             continue
                            
#                     if not selected:
#                         print(gal['MASS_MED_massrun'])
#                         best_boxes_nuvrj = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'.npy')
#                 else:
#                     best_boxes_nuvrj = np.load('best_select_box_lepharecheck_'+diagram_type+'_'+phot_config+'_'+sfq_fit_type+'.npy')
                    
#                 if abs(gal['MAG_ABS_r']) < 50. and abs(gal['MAG_ABS_J'])< 50. and gal['MAG_ABS_NUV']< 50. and abs(gal['MAG_ABS_r'])< 50.:
#                     rj = gal['MAG_ABS_r'] - gal['MAG_ABS_J']
#                     nuvr = gal['MAG_ABS_NUV'] - gal['MAG_ABS_r']
#                     if gal[z_keyname]>0.2 and gal[z_keyname]<0.5:
#                         delta_gv_nuvrj = nuvr - gv_boundary_2(rk,best_boxes_nuvrj[0][0],best_boxes_nuvrj[0][1],best_boxes_nuvrj[0][2])
#                         if delta_gv_nuvrj<0:
#                             sfq_nuvrj.append(1.)
#                         else:
#                             sfq_nuvrj.append(0.)
#                     elif gal[z_keyname]>0.5 and gal[z_keyname]<0.8:
#                         delta_gv_nuvrj = nuvr - gv_boundary_2(rj,best_boxes_nuvrj[1][0],best_boxes_nuvrj[1][1],best_boxes_nuvrj[1][2])
#                         if delta_gv_nuvrj<0:
#                             sfq_nuvrj.append(1.)
#                         else:
#                             sfq_nuvrj.append(0.)
#                     elif gal[z_keyname]>0.8 and gal[z_keyname]<1.1:
#                         delta_gv_nuvrj = nuvr - gv_boundary_2(rj,best_boxes_nuvrj[2][0],best_boxes_nuvrj[2][1],best_boxes_nuvrj[2][2])
#                         if delta_gv_nuvrj<0:
#                             sfq_nuvrj.append(1.)
#                         else:
#                             sfq_nuvrj.append(0.)
#                     elif gal[z_keyname]>1.1 and gal[z_keyname]<1.5:
#                         delta_gv_nuvrj = nuvr - gv_boundary_2(rj,best_boxes_nuvrj[3][0],best_boxes_nuvrj[3][1],best_boxes_nuvrj[3][2])
#                         if delta_gv_nuvrj<0:
#                             sfq_nuvrj.append(1.)
#                         else:
#                             sfq_nuvrj.append(0.)
#                     else:
#                         sfq_nuvrj.append(-99.)
#                 else:
#                     sfq_nuvrj.append(99.)
                
#             sfq_col_nuvrj = Column(name='sfq_nuvrj_myrun_'+sfq_fit_type,data=sfq_nuvrk)  # 1=sf, 0=q
#             if 'sfq_nuvrj_myrun'+sfq_fit_type in cat.keys():
#                 cat.remove_column('sfq_nuvrj_myrun'+sfq_fit_type)
#             cat.add_column(sfq_col_nuvrj)
        
        # write catalog
        cat.write(cat_full_name.replace('.fits','_sfq_added.fits'),overwrite=True)
        